# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [11]:
import numpy as np
import pandas as pd
import patsy
import statsmodels.formula.api as smf

df['log_renda'] = np.log(df['renda'])

cat_vars = df.select_dtypes(include=['object', 'category']).columns

for var in cat_vars:
    df[var] = df[var].astype('category')
    most_freq = df[var].value_counts().idxmax()
    df[var] = df[var].cat.set_categories(
        [most_freq] + [cat for cat in df[var].cat.categories if cat != most_freq],
        ordered=False
    )

formula = 'log_renda ~ ' + ' + '.join([col for col in df.columns if col not in ['renda', 'log_renda']])

y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

modelo = sm.OLS(y, X).fit()

print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     99.56
Date:                Mon, 30 Jun 2025   Prob (F-statistic):               0.00
Time:                        00:17:24   Log-Likelihood:                -13196.
No. Observations:               12466   AIC:                         2.647e+04
Df Residuals:                   12425   BIC:                         2.678e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [14]:
modelo = smf.ols(formula=formula, data=df).fit()

formula = formula.replace(' + qtd_filhos', '') 
modelo_ajustado = smf.ols(formula=formula, data=df).fit()

print(modelo_ajustado.summary())




                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     102.1
Date:                Mon, 30 Jun 2025   Prob (F-statistic):               0.00
Time:                        00:19:17   Log-Likelihood:                -13196.
No. Observations:               12466   AIC:                         2.647e+04
Df Residuals:                   12426   BIC:                         2.677e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [15]:
modelo.params[['sexo[T.M]', 'tipo_renda[T.Empresário]', 'educacao[T.Pós graduação]']]
modelo_ajustado.params[['sexo[T.M]', 'tipo_renda[T.Empresário]', 'educacao[T.Pós graduação]']]


sexo[T.M]                    0.481214
tipo_renda[T.Empresário]     0.179233
educacao[T.Pós graduação]    0.525339
dtype: float64

In [ ]:
O modelo não melhorou nem piorou.

In [17]:
import statsmodels.formula.api as smf

def backward_selection(data, formula, signif_level=0.05):
    modelo = smf.ols(formula=formula, data=data).fit()
    
    while True:
        pvalues = modelo.pvalues.drop('Intercept')
        max_pval = pvalues.max()
        
        if max_pval < signif_level:
            break
        
        var_to_remove = pvalues.idxmax()
        print(f"Removendo variável '{var_to_remove}' com p-valor = {max_pval:.4f}")
        
        base_var = var_to_remove.split('[')[0]
        
        termos = formula.split('~')[1].strip().split('+')
        termos = [t.strip() for t in termos]
        
        termo_remover = None
        for t in termos:
            if t.startswith(base_var):
                termo_remover = t
                break
        
        if termo_remover is None:
            print(f"Aviso: termo para remover '{base_var}' não encontrado na fórmula.")
            break
        
        termos.remove(termo_remover)
        
        formula = 'log_renda ~ ' + ' + '.join(termos)
        modelo = smf.ols(formula=formula, data=data).fit()
        
    return modelo, formula

formula_inicial = 'log_renda ~ ' + ' + '.join([col for col in df.columns if col not in ['renda', 'log_renda']])
modelo_final, formula_final = backward_selection(df, formula_inicial)

print("\nModelo final:")
print(modelo_final.summary())

modelo_inicial = smf.ols(formula=formula_inicial, data=df).fit()

print(f"\nIndicadores:\n")
print(f"Modelo inicial - R² ajustado: {modelo_inicial.rsquared_adj:.4f}, AIC: {modelo_inicial.aic:.2f}, BIC: {modelo_inicial.bic:.2f}")
print(f"Modelo final   - R² ajustado: {modelo_final.rsquared_adj:.4f}, AIC: {modelo_final.aic:.2f}, BIC: {modelo_final.bic:.2f}")


Removendo variável 'qtd_filhos' com p-valor = 0.9655
Removendo variável 'posse_de_veiculo[T.True]' com p-valor = 0.9473
Removendo variável 'data_ref[T.2015-11-01]' com p-valor = 0.8800
Removendo variável 'tipo_renda[T.Bolsista]' com p-valor = 0.9146
Removendo variável 'tipo_residencia[T.Comunitário]' com p-valor = 0.8369
Removendo variável 'educacao[T.Superior incompleto]' com p-valor = 0.6263
Removendo variável 'mau[T.True]' com p-valor = 0.6266
Removendo variável 'estado_civil[T.União]' com p-valor = 0.3310
Removendo variável 'index' com p-valor = 0.2004
Removendo variável 'qt_pessoas_residencia' com p-valor = 0.0771

Modelo final:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     907.7
Date:                Mon, 30 Jun 2025   Pr

In [ ]:
Houve uma pequena redução na qualidade do ajuste (R² ajustado caiu um pouco e AIC subiu). O BIC, que penaliza modelos maiores, melhorou.

